### **Gaussian Splatting for Hayden Library Lantern**

In [ ]:
!git clone https://github.com/ekagra1602/gaussian-splatting-spectral
%cd gaussian-splatting-spectral

In [ ]:
!pip install -r requirements.txt
!apt-get -qq update && apt-get -qq install -y ffmpeg

**Prep Dataset**

In [ ]:
Renew Dataset
!rm -rf {/content/gaussian-splatting-spectral/lantern_ds_full}

In [ ]:
%cd /content/gaussian-splatting-spectral
!python scripts/dataset_prep.py \
  --video /content/Lantern_Video_From_Above.mp4 \
  --out lantern_ds_full \
  --target_frames 70 \
  --min_sharpness 70 \
  --width 1600

## **Run VGGT to predict camera poses, depth**

In [ ]:
!git clone https://github.com/facebookresearch/vggt.git /content/vggt
%cd /content/vggt
!python -c "import vggt; print('VGGT installed!')"

In [ ]:
pip install -r requirements_demo.txt

In [ ]:
!python /content/vggt/demo_colmap.py \
  --scene_dir=/content/gaussian-splatting-spectral/lantern_ds_full \
  --use_ba \
  --query_frame_num=8 \
  --max_query_pts=3072 \
  --fine_tracking \
  --vis_thresh=0.2

## **Restore Session**

In [ ]:
# restore_session.py - Start of every new session
import os
from google.colab import drive

print("🔄 Restoring session...")

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Smart repo handling
REPO_URL = "https://github.com/ekagra1602/gaussian-splatting-spectral"
REPO_DIR = "/content/gaussian-splatting-spectral"

if not os.path.exists(REPO_DIR):
    print("📥 Cloning repository (first time)...")
    !git clone {REPO_URL}
else:
    print("✅ Repository exists, pulling latest changes...")
    %cd {REPO_DIR}
    !git pull origin main || echo "⚠️  Pull failed (maybe no internet or no changes)"

%cd {REPO_DIR}

# 3. Restore dataset from Google Drive backup
print("📦 Restoring lantern_ds dataset...")
if os.path.exists('/content/drive/MyDrive/GaussianSplatting_Backup/lantern_ds'):
    !cp -r /content/drive/MyDrive/GaussianSplatting_Backup/lantern_ds {REPO_DIR}/
    print("✅ Dataset restored from Drive")
else:
    print("❌ Backup not found in Drive! Check your backup location.")

# 4. Reinstall dependencies
print("📚 Installing dependencies...")
!pip install -q hydra-core omegaconf pycolmap==3.10.0

# 5. Comprehensive verification
print("\n🔍 Verification:")
checks = {
    "Repository": os.path.exists(REPO_DIR),
    "lantern_ds": os.path.exists(f"{REPO_DIR}/lantern_ds"),
    "images": os.path.exists(f"{REPO_DIR}/lantern_ds/images"),
    "sparse": os.path.exists(f"{REPO_DIR}/lantern_ds/sparse"),
    "COLMAP files": os.path.exists(f"{REPO_DIR}/lantern_ds/sparse/cameras.bin")
}

for item, exists in checks.items():
    status = "✅" if exists else "❌"
    print(f"  {status} {item}")

if checks["sparse"]:
    sparse_files = os.listdir(f"{REPO_DIR}/lantern_ds/sparse/")
    print(f"\n📁 Sparse folder contains: {sparse_files}")

print(f"\n📂 Working directory: {os.getcwd()}")
print("\n🚀 Session fully restored! Ready to continue.")

### Backup Session

In [ ]:
# 💾 BACKUP CELL
from google.colab import drive
import os

# Mount Drive (safe to run multiple times)
drive.mount('/content/drive')

WORK_DIR = "/content/GaussianSplatting/lantern_ds"
BACKUP_DIR = "/content/drive/MyDrive/gaussian-splatting-spectral_backup"

print("💾 Starting backup...")

# Create backup directory
!mkdir -p {BACKUP_DIR}

# Sync with progress
!rsync -av --progress {WORK_DIR}/ {BACKUP_DIR}/lantern_ds/

# Show what was backed up
print("\n📊 Backup Summary:")
!du -sh {BACKUP_DIR}/lantern_ds/
print(f"📁 Backed up to: {BACKUP_DIR}/lantern_ds/")

# List major folders
print("\n📂 Backed up folders:")
for folder in ['images', 'sparse', 'outputs', 'checkpoints']:
    folder_path = f"{BACKUP_DIR}/lantern_ds/{folder}"
    if os.path.exists(folder_path):
        size = !du -sh {folder_path} | cut -f1
        print(f"  ✅ {folder}: {size[0]}")
    else:
        print(f"  ⚪ {folder}: (not yet created)")

print("\n✅ Backup complete! Safe to disconnect.")

# **Run Gaussian Spplatting on output from VGGT**

In [ ]:
# Install gsplat (recommended version from VGGT docs)
!pip install gsplat

# Install additional dependencies
!pip install \
  opencv-python \
  pillow \
  tqdm \
  lpips \
  trimesh \
  plyfile

print("✅ gsplat installed!")

In [ ]:
# Clone gsplat repo to get example training scripts
%cd /content
!git clone https://github.com/nerfstudio-project/gsplat.git
%cd gsplat

# Install example requirements
!pip install -r examples/requirements.txt

print("✅ gsplat repo cloned!")

In [ ]:
#Create 0 directory
import os
import shutil

# Create the 0 subdirectory
sparse_dir = "/content/gaussian-splatting-spectral/lantern_ds_full/sparse"
sparse_0_dir = f"{sparse_dir}/0"

os.makedirs(sparse_0_dir, exist_ok=True)

# Move all .bin and .ply files into 0/
for file in os.listdir(sparse_dir):
    if file.endswith(('.bin', '.ply')):
        src = f"{sparse_dir}/{file}"
        dst = f"{sparse_0_dir}/{file}"
        shutil.move(src, dst)
        print(f"Moved: {file} → sparse/0/{file}")

print("\n✅ Dataset structure fixed!")

In [ ]:
!echo "=== Checking dataset structure ==="
!ls -lh /content/gaussian-splatting-spectral/lantern_ds/
!echo ""
!ls -lh /content/gaussian-splatting-spectral/lantern_ds/sparse/0/
!echo ""
!ls /content/gaussian-splatting-spectral/lantern_ds/images/ | head -5

### **Ignore**

In [ ]:
# Create the missing __init__.py file
!touch /content/gsplat/examples/datasets/__init__.py

print("Created __init__.py in datasets/")

In [ ]:
# Uninstall old version
!pip uninstall -y gsplat

# Install latest version
!pip install gsplat

print("✅ gsplat upgraded to latest version")

In [ ]:
%cd /content/gsplat/examples

!python simple_trainer.py default \
  --data_dir /content/GaussianSplatting/lantern_ds \
  --result_dir /content/GaussianSplatting/results/lantern_baseline \
  --data_factor 1 \
  --max_steps 10000 \
  --save_ply

In [ ]:
!echo "=== PLY Files ==="
!ls -lh /content/GaussianSplatting/results/lantern_baseline/ply/

!echo ""
!echo "=== Rendered Videos ==="
!ls -lh /content/GaussianSplatting/results/lantern_baseline/videos/

In [ ]:
#Download PLY File
from google.colab import files
files.download("/content/GaussianSplatting/results/lantern_spectral_30k/final.pt")

In [ ]:
%cd /content/gsplat

!python examples/simple_trainer.py default \
  --data_dir /content/gaussian-splatting-spectral/lantern_ds_full \
  --result_dir /content/results/lantern_baseline \
  --data-factor 1 \
  --max-steps 30000 \
  --save-ply \
  --disable-viewer \
  --sh-degree 3 \
  --sh-degree-interval 1000 \
  --init-opa 0.1 \
  --init-scale 1.0 \
  --ssim-lambda 0.2 \
  --random-bkgd \
  --strategy.refine-start-iter 500 \
  --strategy.refine-stop-iter 15000 \
  --strategy.reset-every 3000 \
  --strategy.refine-every 100 \
  --strategy.prune-opa 0.005 \
  --strategy.grow-grad2d 0.0002 \
  --strategy.grow-scale3d 0.01 \
  --strategy.prune-scale3d 0.1 \
  --strategy.prune-scale2d 0.15

/content/gsplat
2025-10-16 19:34:01.592801: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 19:34:01.611191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760643241.635627   14148 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760643241.642370   14148 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760643241.659462   14148 computation_placer.cc:177] computation placer already registered. Please check linkage an

In [ ]:
%cd /content/gsplat

# See parameters for the 'default' config
!python examples/simple_trainer.py default --help

### ***Run Custom Script***

In [ ]:
# ============================================================
# CELL 1: Install Spectral-GS Dependencies
# ============================================================
!pip install scipy
print("✅ Spectral-GS dependencies installed!")

In [ ]:
# ============================================================
# CELL 2: Verify Spectral-GS Installation
# ============================================================
import sys
sys.path.insert(0, '/content/gaussian-splatting-spectral/src')

try:
    from spectral_gs import compute_spectral_entropy, SpectralStrategy, apply_view_consistent_filter
    print("✅ Spectral-GS modules imported successfully!")

    # Quick test
    import torch
    test_scales = torch.randn(10, 3).abs() + 0.1
    test_quats = torch.randn(10, 4)
    test_quats = test_quats / torch.norm(test_quats, dim=-1, keepdim=True)

    entropy = compute_spectral_entropy(test_scales, test_quats)
    print(f"   Test entropy computation: mean = {entropy.mean():.3f}")
    print("   ✓ All components working!")

except ImportError as e:
    print(f"❌ Import failed: {e}")
    print("\n📝 Troubleshooting:")
    print("1. Make sure you pushed spectral_gs code to your GitHub repo")
    print("2. Run 'git pull' to update your Colab repository")
    print("3. Check that /content/gaussian-splatting-spectral/src/spectral_gs/ exists")

In [ ]:
  # Uninstall current pycolmap
  !pip uninstall pycolmap -y

  # Install the EXACT version gsplat needs (from their requirements.txt)
  !pip install git+https://github.com/rmbrualla/pycolmap@cc7ea4b7301720ac29287dbe450952511b32125e

  # Verify it worked
  !python -c "from pycolmap import SceneManager; print('✅ SceneManager imported successfully')"

In [ ]:
# ============================================================
# CELL 3: Train with Spectral-GS (MAIN TRAINING)
# ============================================================
# FIRST: Create the missing __init__.py
!touch /content/gsplat/examples/datasets/__init__.py

# Copy updated training script
import shutil
shutil.copy(
    '/content/gaussian-splatting-spectral/scripts/train_spectral_gs.py',
    '/content/gsplat/examples/train_spectral_gs.py'
)

# Copy spectral_gs module
shutil.copytree(
    '/content/gaussian-splatting-spectral/src/spectral_gs',
    '/content/gsplat/examples/spectral_gs',
    dirs_exist_ok=True
)

print("✅ All files copied!")

%cd /content

import torch, gc
torch.cuda.empty_cache()
gc.collect()
# 3. Test with spectral splitting (now properly implemented!)
!python gsplat/examples/train_spectral_gs.py \
  --data_dir /content/gaussian-splatting-spectral/lantern_ds_full \
  --result_dir /content/results/lantern_spectral\
  --max_steps 10000 \
  --enable_spectral_splitting \
  --spectral_threshold 0.5 \
  --log_every 100 \
  --verbose


✅ All files copied!
/content
2025-10-16 19:29:40.115607: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-16 19:29:40.133192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760642980.155617   13015 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760642980.162299   13015 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760642980.179265   13015 computation_placer.cc:177] computation placer already registered. Please che

In [ ]:
#Download PLY File
from google.colab import files
files.download("/content/gaussian-splatting-spectral/results/lantern_spectral/final.ply")

In [ ]:

# ============================================================
# CELL 4: Check Results
# ============================================================
import os

result_dir = "/content/GaussianSplatting/results/lantern_spectral_30k"

print("📊 Training Results\n")
print("=" * 60)

if os.path.exists(result_dir):
    # List all files
    files = os.listdir(result_dir)

    # Count checkpoints
    checkpoints = [f for f in files if f.endswith('.pt')]
    print(f"✅ Checkpoints saved: {len(checkpoints)}")

    # Check for final model
    if os.path.exists(f"{result_dir}/final.pt"):
        size = os.path.getsize(f"{result_dir}/final.pt") / (1024*1024)
        print(f"✅ Final model: {size:.1f} MB")

    if os.path.exists(f"{result_dir}/final.ply"):
        size = os.path.getsize(f"{result_dir}/final.ply") / (1024*1024)
        print(f"✅ Final PLY: {size:.1f} MB")

    print(f"\n📁 Full results at: {result_dir}")
else:
    print("❌ Results directory not found!")

print("=" * 60)

In [ ]:
# ============================================================
# CELL 5: Download Results
# ============================================================
from google.colab import files

# Download final PLY file for viewing
ply_path = "/content/GaussianSplatting/results/lantern_spectral_30k/final.ply"

if os.path.exists(ply_path):
    print(f"📥 Downloading: {ply_path}")
    files.download(ply_path)
    print("✅ Download complete!")
else:
    print(f"❌ PLY file not found at: {ply_path}")
    print("\nAvailable files:")
    !ls -lh /content/GaussianSplatting/results/lantern_spectral_30k/

In [ ]:
# ============================================================
# CELL 6: Backup to Google Drive (RECOMMENDED)
# ============================================================
from google.colab import drive

# Mount Drive if not already mounted
drive.mount('/content/drive')

BACKUP_DIR = "/content/drive/MyDrive/GaussianSplatting_Backup"
SOURCE_DIR = "/content/GaussianSplatting/results"

print("💾 Backing up results to Google Drive...")
!mkdir -p {BACKUP_DIR}
!rsync -av --progress {SOURCE_DIR}/ {BACKUP_DIR}/results/

print("\n✅ Backup complete!")
print(f"📁 Saved to: {BACKUP_DIR}/results/")